In [18]:
# DEPENDENCIES
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd

In [2]:
# NASA MARS NEWS

# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

content_title = soup.find(class_="content_title").text.replace('\n', '')
print("Title:" , content_title)
content_paragraph = soup.find(class_="rollover_description_inner").text.replace('\n', '')
print("Paragraph:" , content_paragraph)

Title: Opportunity Hunkers Down During Dust Storm
Paragraph: As of Tuesday morning, June 19, the Martian dust storm had grown in size and was officially a "planet-encircling" (or "global") dust event. 


In [3]:
# JPL MARS FEATURED IMAGE
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = bs(html, 'html.parser')

# Find desired info
footer = soup.find('footer')
img = footer.find('a')
link = img['data-fancybox-href']
featured_image_url = 'https://www.jpl.nasa.gov' + link
print(featured_image_url)

browser.quit()

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16763_ip.jpg


In [4]:
# MARS WEATHER

# URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'html.parser')

# reset variable in case of multiple runs
mars_weather = 'a'

# loop with break point after most recent tweet by @MarsWxReport
for x in range(1,2):
    tweets = soup.find_all(class_="tweet")
    for tweet in tweets:
        tweet_id = tweet['data-tweet-id']
        tweet_user = tweet['data-screen-name']
        #ensure scraped tweet was authored by this account and not retweeted
        if tweet_user == 'MarsWxReport':
            mars_weather = tweet.find(class_="content").find(class_="js-tweet-text-container").find('p').text
            break
            
print(mars_weather)

Sol 2108 (2018-07-12), Sunny, high -24C/-11F, low -65C/-84F, pressure at 8.06 hPa, daylight 05:19-17:27


In [20]:
# MARS FACTS

# scrape page for tables
url = 'https://space-facts.com/mars/'
table = pd.read_html(url)

# Convert table to DataFrame 
df = table[0]
df = df.transpose()
df.columns = df.iloc[0].str.replace(":", "")
df = df.drop(0)

# Convert DataFrame to HTML table
html_table = df.to_html().replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Equatorial Diameter</th>      <th>Polar Diameter</th>      <th>Mass</th>      <th>Moons</th>      <th>Orbit Distance</th>      <th>Orbit Period</th>      <th>Surface Temperature</th>      <th>First Record</th>      <th>Recorded By</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>6,792 km</td>      <td>6,752 km</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>      <td>2 (Phobos &amp; Deimos)</td>      <td>227,943,824 km (1.52 AU)</td>      <td>687 days (1.9 years)</td>      <td>-153 to 20 °C</td>      <td>2nd millennium BC</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [17]:
# MARS HEMISPHERES
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

# Initialize Lists
hemisphere_image_urls = []
img_url = []
title = []
links_to_visit = []

# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup = bs(html, 'html.parser')

# Find desired info
descriptions = soup.find_all(class_='description')
for description in descriptions: 
    link = description.find('a')
    h3 = link.find('h3')
    partial_text = h3.text
    links_to_visit.append(partial_text)

for link in links_to_visit:
    add_title = link.replace(" Enhanced", "")
    title.append(add_title)
    browser.click_link_by_partial_text(link)
    html = browser.html
    soup = bs(html, 'html.parser')
    download = soup.find(class_='downloads')
    ul = download.find('ul')
    li = ul.find('li')
    lnk = li.find('a')
    add_url = lnk['href']
    img_url.append(add_url)
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')

browser.quit()

dict_0 = {"title": title[0], "img_url": img_url[0]}
dict_1 = {"title": title[1], "img_url": img_url[1]}
dict_2 = {"title": title[2], "img_url": img_url[2]}
dict_3 = {"title": title[3], "img_url": img_url[3]}
hemisphere_image_urls.append([dict_0,dict_1,dict_2,dict_3])
hemisphere_image_urls

[[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere'},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere'}]]